<a href="https://colab.research.google.com/github/S-Kia/1st-Kibo-RPC_won-spaceY/blob/master/Backward_optimize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
from multiprocessing import Pool, Manager

# Set the start and end date
start_date = "2023-12-13"
end_date = "2023-12-16"

# Get historical hourly data for BTC, BNB, and USDT
btc_data = yf.download("BTC-USD", start=start_date, end=end_date, interval="1h")
eth_data = yf.download("ETH-USD", start=start_date, end=end_date, interval="1h")
bnb_data = yf.download("BNB-USD", start=start_date, end=end_date, interval="1h")
xrp_data = yf.download("XRP-USD", start=start_date, end=end_date, interval="1h")
sol_data = yf.download("SOL-USD", start=start_date, end=end_date, interval="1h")
ada_data = yf.download("ADA-USD", start=start_date, end=end_date, interval="1h")
doge_data = yf.download("DOGE-USD", start=start_date, end=end_date, interval="1h")
usdt_data = yf.download("USDT-USD", start=start_date, end=end_date, interval="1h")

# Define the range of values for a-h and threshold
percentage_range = [0,20,40,60,80,100]
threshold_range = [0.5,1,2,5]

# Use Manager to create a shared list for top_configurations
manager = Manager()
top_configurations = manager.list()

# Function to calculate ROI for a specific combination of values
def calculate_roi(combination):
    a, b, c, d, e, f, g, h, threshold = combination
    # Initialize portfolio
    percentage = {'BTC': a/100, 'ETH': b/100, 'BNB': c/100, 'XRP': d/100, 'SOL': e/100, 'ADA': f/100, 'DOGE': g/100, 'USDT': h/100}
    initial_investment = 100  # in USD
    current_balance = initial_investment
    portfolio = {coin: value * initial_investment for coin, value in percentage.items()}
    # Find number of each coin
    portfolio['BTC'] /= btc_data['Close'][1]
    portfolio['ETH'] /= eth_data['Close'][1]
    portfolio['BNB'] /= bnb_data['Close'][1]
    portfolio['XRP'] /= xrp_data['Close'][1]
    portfolio['SOL'] /= sol_data['Close'][1]
    portfolio['ADA'] /= ada_data['Close'][1]
    portfolio['DOGE'] /= doge_data['Close'][1]
    #print(portfolio)

    # Rebalance function
    def rebalance(portfolio, prices):
        total_value = sum(portfolio[coin] * prices[coin] for coin in portfolio)
        for coin in portfolio:
            target_value = total_value * percentage[coin]
            target_amount = target_value / prices[coin]
            trade_amount = target_amount - portfolio[coin]
            trade_value = trade_amount * prices[coin]
            portfolio[coin] = target_amount
        return portfolio

    # Iterate through the data
    j = 0
    for i in range(1, len(btc_data)):
        # Calculate percentage change
        btc_change = (btc_data['Close'][i] - btc_data['Close'][j]) / btc_data['Close'][j] * 100
        eth_change = (eth_data['Close'][i] - eth_data['Close'][j]) / eth_data['Close'][j] * 100
        bnb_change = (bnb_data['Close'][i] - bnb_data['Close'][j]) / bnb_data['Close'][j] * 100
        xrp_change = (xrp_data['Close'][i] - xrp_data['Close'][j]) / xrp_data['Close'][j] * 100
        sol_change = (sol_data['Close'][i] - sol_data['Close'][j]) / sol_data['Close'][j] * 100
        ada_change = (ada_data['Close'][i] - ada_data['Close'][j]) / ada_data['Close'][j] * 100
        doge_change = (doge_data['Close'][i] - doge_data['Close'][j]) / doge_data['Close'][j] * 100
        usdt_change = (usdt_data['Close'][i] - usdt_data['Close'][j]) / usdt_data['Close'][j] * 100

        # Check if any coin's percentage change is greater than the threshold
        if (percentage['BTC'] != 0 and abs(btc_change) > threshold) or \
            (percentage['ETH'] != 0 and abs(eth_change) > threshold) or \
            (percentage['BNB'] != 0 and abs(bnb_change) > threshold) or \
            (percentage['XRP'] != 0 and abs(xrp_change) > threshold) or \
            (percentage['SOL'] != 0 and abs(sol_change) > threshold) or \
            (percentage['ADA'] != 0 and abs(ada_change) > threshold) or \
            (percentage['DOGE'] != 0 and abs(doge_change) > threshold) or \
            (percentage['USDT'] != 0 and abs(usdt_change) > threshold):
            coin_prices = {'BTC': btc_data['Close'][i],
                            'ETH': eth_data['Close'][i],
                            'BNB': bnb_data['Close'][i],
                            'XRP': xrp_data['Close'][i],
                            'SOL': sol_data['Close'][i],
                            'ADA': ada_data['Close'][i] ,
                            'DOGE': doge_data['Close'][i] ,
                            'USDT': usdt_data['Close'][i]}

            j = i
            portfolio = rebalance(portfolio, coin_prices)
            portfolio_price = {coin: portfolio[coin] * coin_prices[coin] for coin in portfolio}
            Last_value = sum(portfolio_price[coin] for coin in portfolio_price)

            # Calculate ROI
            roi = (Last_value - initial_investment) / initial_investment * 100

        else:
          roi = 0

        # Update top configurations
        top_configurations.append({'BTC': a, 'ETH': b, 'BNB': c, 'XRP': d, 'SOL': e, 'ADA': f, 'DOGE': g, 'USDT': h, 'threshold': threshold, 'ROI': roi})

    return {'BTC': a, 'ETH': b, 'BNB': c, 'XRP': d, 'SOL': e, 'ADA': f, 'DOGE': g, 'USDT': h, 'threshold': threshold, 'ROI': roi}

# Generate all combinations of values
combinations = [(a, b, c, d, e, f, g, h, threshold) for a in percentage_range
                                                        for b in percentage_range
                                                        for c in percentage_range
                                                        for d in percentage_range
                                                        for e in percentage_range
                                                        for f in percentage_range
                                                        for g in percentage_range
                                                        for h in percentage_range
                                                        for threshold in threshold_range
                if a + b + c + d + e + f + g + h == 100]

# Number of processes to use (adjust as needed)
num_processes = 4

# Create a context manager for the Pool to ensure proper cleanup
with Pool(num_processes) as pool:
    try:
        # Calculate ROI for all combinations in parallel
        pool.map(calculate_roi, combinations)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # Close the pool to release resources
        pool.close()
        pool.join()

# Convert the shared list to a regular list for further processing if needed
top_configurations = list(top_configurations)
print(len(top_configurations))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
An error occurred: index 71 is out of bounds for axis 0 with size 71
1120


In [ ]:
# Filter data to keep dictionaries with the maximum 'ROI' value for each 'sym' combination
max_roi = {}
for item in top_configurations:
    sym_key = (item['BTC'], item['ETH'], item['BNB'], item['XRP'], item['SOL'], item['ADA'], item['DOGE'], item['USDT'], item['threshold'])
    if sym_key not in max_roi or item['ROI'] > max_roi[sym_key]['ROI']:
        max_roi[sym_key] = item

# Sort top configurations based on ROI and keep only the top
filtered_data = list(max_roi.values())
top_configurations = sorted(filtered_data, key=lambda x: x['ROI'], reverse=True)[:50]

# Print the top configurations and their ROI
print("Top 5 Configurations:")
for i, config in enumerate(top_configurations):
    print(f"{i + 1}. Configuration: {config}, ROI: {config['ROI']:.2f}%")

Top 5 Configurations:
1. Configuration: {'BTC': 20, 'ETH': 0, 'BNB': 0, 'XRP': 0, 'SOL': 60, 'ADA': 20, 'DOGE': 0, 'USDT': 0, 'threshold': 0.5, 'ROI': 14.232494320874991}, ROI: 14.23%
2. Configuration: {'BTC': 0, 'ETH': 0, 'BNB': 0, 'XRP': 0, 'SOL': 60, 'ADA': 20, 'DOGE': 0, 'USDT': 20, 'threshold': 2, 'ROI': 13.10827305427037}, ROI: 13.11%
3. Configuration: {'BTC': 0, 'ETH': 40, 'BNB': 0, 'XRP': 0, 'SOL': 40, 'ADA': 20, 'DOGE': 0, 'USDT': 0, 'threshold': 0.5, 'ROI': 11.62546811532279}, ROI: 11.63%
4. Configuration: {'BTC': 0, 'ETH': 0, 'BNB': 0, 'XRP': 40, 'SOL': 0, 'ADA': 40, 'DOGE': 20, 'USDT': 0, 'threshold': 0.5, 'ROI': 9.33110726686985}, ROI: 9.33%
5. Configuration: {'BTC': 0, 'ETH': 0, 'BNB': 20, 'XRP': 0, 'SOL': 20, 'ADA': 40, 'DOGE': 0, 'USDT': 20, 'threshold': 2, 'ROI': 8.964716366613189}, ROI: 8.96%
6. Configuration: {'BTC': 20, 'ETH': 20, 'BNB': 0, 'XRP': 0, 'SOL': 0, 'ADA': 20, 'DOGE': 40, 'USDT': 0, 'threshold': 0.5, 'ROI': 7.596518737308684}, ROI: 7.60%
7. Configuration: